In [116]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, DepthwiseConv2D, Input, add
from keras.models import Sequential, Model
from keras import optimizers
import keras.backend as K
import keras
import tensorflow as tf

def _depthwise_conv_block(inputs, pointwise_conv_filters, depth_multiplier=1, strides=(1, 1), block_id=1, activation="relu"):
    x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=2,strides=strides, use_bias = True, activation=activation, name = 'block_%d_conv_dw' % block_id)(inputs)
    x = Conv2D(pointwise_conv_filters, kernel_size=(1, 1),padding = "same",use_bias=True, strides=strides, name = 'block_%d_conv_pw' % block_id)(x)
    x = add([inputs, x])
    return x

# # 构造模型；
# model = Sequential()
# # 加卷积层，activation在卷积神经网络中基本上都用relu
# model.add(DepthwiseConv2D((3, 3),padding='same',depth_multiplier=2,strides= (1, 1),use_bias = True, input_shape=(160, 120, 1)))
input_shape=(120, 160, 1)
input_tensor = Input(shape = input_shape)
net = {}

x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=8,strides= (1, 1),use_bias = True, activation='relu')(input_tensor)
x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides= (2, 2),use_bias = True)(x)
x = Conv2D(8, (1,1),padding='same',use_bias=True,strides=(1, 1))(x)
x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=2,strides= (2, 2),use_bias = True)(x)
x = Conv2D(8, (1,1),padding='same',use_bias=True,strides=(1, 1), name="blockstart")(x)

# block1
x = _depthwise_conv_block(x, 8, 1, block_id=1)
# block2
x = _depthwise_conv_block(x, 8, 1, block_id=2)
# block3
x = _depthwise_conv_block(x, 8, 1, block_id=3)  # 30 * 40 * 8

x = Conv2D(16, (1,1),padding='same',use_bias=True,strides=(1, 1))(x)  # 30 * 40 * 16
x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides= (1, 1),use_bias = True)(x)  # 30 * 40 * 16
x = Conv2D(16, (1,1),padding='same',use_bias=True,strides=(1, 1))(x)  # 30 * 40 * 16
x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=2,strides= (2, 2),use_bias = True)(x)  # 15 * 20 * 32
x = Conv2D(24, (1,1),padding='same',use_bias=True,strides=(1, 1))(x)  # 15 * 20 * 24

# block4~6
x = _depthwise_conv_block(x, 24, 1, block_id=4)  # 15*20*24
x = _depthwise_conv_block(x, 24, 1, block_id=5)  # 15*20*24
x = _depthwise_conv_block(x, 24, 1, block_id=6)  # 15*20*24

x = Conv2D(48, (1,1),padding='same',use_bias=True,strides=(1, 1), activation="relu", name='Conv2D_layer11')(x)
# --> 15 * 20 * 48
x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides= (1, 1),use_bias = True, activation="relu", name='DepthWiseConv2D_layer9')(x)
# --> 15 * 20 * 64
x = Conv2D(64, (1,1),padding='same',use_bias=True,strides=(1, 1), activation="relu", name='Conv2D_layer12')(x)

x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides= (1, 1),use_bias = True, activation="relu", name='DepthWiseConv2D_layer14')(x)
# --> 8 * 10 * 40
x = Conv2D(40, (1,1),padding='same',use_bias=True,strides=(2, 2), activation="relu", name='Conv2D_layer15')(x)
# block 7_1 ~7_2
x = _depthwise_conv_block(x, 40, 1, block_id=7)
x = _depthwise_conv_block(x, 40, 1, block_id=8)

# --> 8 * 10 * 80
x = Conv2D(80, (1,1),padding='same',use_bias=True,strides=(1, 1), activation="relu", name='Conv2D_layer16')(x)
# --> 8 * 10 * 80
x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides= (1, 1),use_bias = True, activation="relu", name='DepthWiseConv2D_layer17')(x)
# --> 8 * 10 * 80
x = Conv2D(80, (1,1),padding='same',use_bias=True,strides=(1, 1), activation="relu", name='Conv2D_layer18')(x)
net['split_layer2'] = x

# --> 4 * 5 * 80
x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides= (2, 2),use_bias = True, activation="relu", name='DepthWiseConv2D_layer19')(x)
# --> 4 * 5 * 80
x = Conv2D(80, (1,1),padding='same',use_bias=True,strides=(1, 1), activation="relu", name='Conv2D_layer20')(x)
    
net['split_layer3'] = x
    
# --> 2 * 3 * 80
x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides= (2, 2),use_bias = True, activation="relu", name='DepthWiseConv2D_layer21')(x)
    
# --> 2 * 3 * 64
x = Conv2D(64, (1,1),padding='same',use_bias=True,strides=(1, 1), activation="relu", name='Conv2D_layer22')(x)
    
net['split_layer4'] = x
    
# --> 1 * 1 * 64
x = DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides= (3, 3),use_bias = True, activation="relu", name='DepthWiseConv2D_layer23')(x)

net['split_layer5'] = x

# end
net['predictions'] = x
model = Model(input_tensor, net['predictions'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_98 (InputLayer)           (None, 120, 160, 1)  0                                            
__________________________________________________________________________________________________
depthwise_conv2d_326 (Depthwise (None, 120, 160, 8)  80          input_98[0][0]                   
__________________________________________________________________________________________________
depthwise_conv2d_327 (Depthwise (None, 60, 80, 8)    80          depthwise_conv2d_326[0][0]       
__________________________________________________________________________________________________
conv2d_214 (Conv2D)             (None, 60, 80, 8)    72          depthwise_conv2d_327[0][0]       
__________________________________________________________________________________________________
depthwise_